In [1]:
import xml.etree.ElementTree as ET

from launch_ros.substitutions import FindPackageShare


pagkage_name = 'testbot_description'
folder_name = 'description'
file_name = 'sensor_lidar.xacro'

fpshare = FindPackageShare(pagkage_name)

path = fpshare.find(package_name=pagkage_name) + '/' + folder_name + '/' + file_name

print(path)

/home/athimeta/CARVER-Savvy/carverabwu/install/testbot_description/share/testbot_description/description/sensor_lidar.xacro


In [2]:
import os

print(os.environ['ABWUDRL_BASE_PATH'])
print(os.environ['GAZEBO_MODEL_PATH'])

print(os.environ['GAZEBO_ROBOT_MODEL_PATH'])
print(os.environ['GAZEBO_PLUGIN_PATH'])


/home/athimeta/CARVER-Savvy/carverabwu
/home/athimeta/CARVER-Savvy/carverabwu/src/abwu_gazebo/models
/home/athimeta/CARVER-Savvy/carverabwu/src/abwu_gazebo/models:/home/athimeta/CARVER-Savvy/carverabwu/src/testbot_description/description
:/home/athimeta/CARVER-Savvy/carverabwu/src/abwu_gazebo/models/turtlebot3_drl_world/obstacle_plugin/lib


In [3]:
import numpy as np
MAX_NUMBER_OBSTACLES = 10
[np.inf] * MAX_NUMBER_OBSTACLES

[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]